In [1]:
import pandas as pd
import numpy as np

In [2]:
NKE = pd.read_csv("NKE_pred.csv")  
JPM = pd.read_csv("JPM_pred.csv")  
SBUX = pd.read_csv("SBUX_pred.csv")  
BRK_B = pd.read_csv("BRK-B_pred.csv")  
RRC = pd.read_csv("RRC_pred.csv")  
CAR = pd.read_csv("CAR_pred.csv")  
COST = pd.read_csv("COST_pred.csv")  

In [11]:
df = pd.merge(NKE, JPM,how='left', on='timestamp')
df = pd.merge(df, SBUX,how='left', on='timestamp')
df = pd.merge(df, BRK_B,how='left', on='timestamp')
df = pd.merge(df, RRC,how='left', on='timestamp')
df = pd.merge(df, CAR,how='left', on='timestamp')
df = pd.merge(df, COST,how='left', on='timestamp')

In [12]:
df

,timestamp,NKE real value,NKE prediction,JPM real value,JPM prediction,SBUX real value,SBUX prediction,BRK-B real value,BRK-B prediction,RRC real value,RRC prediction,CAR real value,CAR prediction,COST real value,COST prediction
0,2019-01-02,74.06,74.190105,99.31,97.644929,64.32,64.422021,202.80,203.158976,9.96,9.624571,22.71,22.840629,204.76,204.986283
1,2019-01-03,72.75,74.240245,97.11,97.669864,61.53,64.444050,191.66,202.143059,9.95,9.679453,21.78,23.207044,200.42,206.270561
2,2019-01-04,74.65,74.290418,100.69,97.694806,63.57,64.466087,195.20,201.132221,10.69,9.734648,23.13,23.579337,206.24,207.562886
3,2019-01-07,75.72,74.340625,100.76,97.719754,63.57,64.488131,196.91,200.126438,11.13,9.790157,23.93,23.957602,207.00,208.863308
4,2019-01-08,76.73,74.390866,100.57,97.744709,63.62,64.510182,196.31,199.125685,11.00,9.845984,24.11,24.341935,208.55,210.171877
5,2019-01-09,76.59,74.441141,100.40,97.769669,63.88,64.532242,196.37,198.129936,11.60,9.902128,25.06,24.732434,209.90,211.488644
6,2019-01-10,76.42,74.491450,100.39,97.794637,64.19,64.554308,196.58,197.139167,11.44,9.958593,25.57,25.129197,210.64,212.813661
7,2019-01-11,76.04,74.541793,99.91,97.819610,63.73,64.576382,196.29,196.153352,11.50,10.015380,26.01,25.532325,210.51,214.146980
8,2019-01-14,76.09,74.592170,100.94,97.844590,63.37,64.598464,195.78,195.172466,11.87,10.072490,26.04,25.941921,209.00,215.488652
9,2019-01-15,77.88,74.642581,101.68,97.869577,64.08,64.620553,196.94,194.196486,11.74,10.129926,25.71,26.358087,211.03,216.838731


In [13]:
df.to_csv('ARIMA+GARCH_preds_3months.csv')

# Baseline MPT

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas_datareader import data as pdr
import fix_yahoo_finance
import datetime
import os
import cvxpy as cp

In [2]:
tickers = ['NKE', 'JPM', 'SBUX', 'BRK-B', 'RRC', 'CAR', 'COST']
risk_free_rate = 0.026 # 1-year T-bill on 1 Jan 2019

In [3]:
def get_adj_closing_prices(tickers, start_date, end_date):
    all_prices = {}
    for ticker in tickers:
        prices = pdr.get_data_yahoo(ticker, 
            start=start_date,
            end=end_date
        )
        all_prices[ticker] = prices["Adj Close"]
    
    return pd.DataFrame(all_prices)

In [4]:
def compute_expected_daily_returns(df):
    return np.array(np.mean(df.T, axis=1))

In [5]:
def compute_daily_covariance_matrix(df): 
    return np.cov(df.T)

In [6]:
def compute_portfolio_expected_return(return_vector, weights):
    """Computes expected return of a portfolio given the asset returns and the asset weights in the portfolio."""
    return return_vector.T.dot(weights)

def compute_portfolio_variance(covariance_matrix, weights):
    """Computes the variance of a portfolio given the asset covariance matrix and the asset weights in the portfolio."""
    return weights.T.dot(covariance_matrix).dot(weights)

def compute_portfolio_std(covariance_matrix, weights):
    """Computes the standard deviation of a portfolio given the asset covariance matrix and the asset weights in the portfolio.""" 
    return np.sqrt(compute_portfolio_variance(covariance_matrix, weights))

In [7]:
 def min_risk_portfolio(expected_returns, covariance_matrix, target_return):
    """Computes the weights of a minimum variance portfolio for a given expected return.
    The portfolio is fully vested (weights sum to 1) and no shorting is allowed.
    
    Args:
        expected_returns (np.array): Expected returns for n assets.
        covariance_matrix (np.array): n x n Covariance matrix of asset returns.
        target_return (float): Expected return target for the portfolio
        
    Returns:
        np.array of length n with the weights of each asset in the portfolio
        OR
        None if no feasible portfolio exists.
    """
    n = expected_returns.shape[0]        #数组的个数
    
    w = cp.Variable(n)                         # Portfolio allocation vector   ?cp.Variable: The optimization variables in a problem.定义问题的变量
    ret = expected_returns.T * w
    risk = cp.quad_form(w,covariance_matrix)  #就是组合方差
    target_ret = cp.Parameter()             #定义问题的参数
    target_ret.value = target_return
    prob = cp.Problem(cp.Minimize(risk),          # Restricting to long-only portfolio
                   [ret == target_ret, # match target_return
                   cp.sum(w) == 1,    # sum of weights in portfolios sum to 1.
                   w >= 0])
    prob.solve()
    
    if prob.status == 'optimal':
        return w.value
    else:
        return None

In [8]:
def efficient_frontier(expected_returns, covariance_matrix):
    """Construct efficient frontier portfolios by a line sweep of target returns.
    
    Args:
        expected_returns (np.array): Expected returns for n assets.
        covariance_matrix (np.array): n x n Covariance matrix of asset returns.
        
    Returns:
        List of np.arrays. Each numpy array in the list represents a portfolio on the efficient frontier.
    
    """
    min_return = np.min(expected_returns)
    max_return = np.max(expected_returns)
    
    target_returns = np.linspace(min_return,max_return, num=200)
    
    portfolio_weights = []
    
    for tr in target_returns:
        result = min_risk_portfolio(expected_returns, covariance_matrix, tr)
        # only add results if optimization was successful
        if result is not None:
            weights = result
            portfolio_weights.append(weights)

    return portfolio_weights

In [9]:
def min_variance_portfolio(covariance_matrix, portfolio_weights):
    """Select minimum variance portfolio from a list of portfolios.
    
    Args:
        covariance_matrix (np.array): n x n Covariance matrix of asset returns.
        portfolio_weights (list of np.array): Each numpy array in the list represents a portfolio.
        
    Returns:
        np.array with weights of portfolio that has the smallest variance.
    """
    portfolio_variances = []

    for weight in portfolio_weights:  
        variance = compute_portfolio_variance(covariance_matrix, weight)
        portfolio_variances.append(variance) 
        
    index = np.argmin(portfolio_variances)   #Returns the indices of the minimum values along an axis.
    
    return portfolio_weights[index]

In [10]:
def max_sharpe_portfolio(expected_returns, covariance_matrix, risk_free_rate, portfolio_weights):
    """Select max sharpe portfolio from a list of portfolios.
    
    Args:
        expected_returns (np.array): Expected returns for n assets.
        covariance_matrix (np.array): n x n Covariance matrix of asset returns.
        risk_free_rate (float): Risk free rate.
        portfolio_weights (list of np.array): Each numpy array in the list represents a portfolio.
        
    Returns:
        np.array with weights of portfolio that maximizes sharpe ratio.  
    """
    sharpe_ratios = []

    for weight in portfolio_weights:  
        mean = compute_portfolio_expected_return(expected_returns, weight)
        std = compute_portfolio_std(covariance_matrix, weight)
        sharpe_ratio = (mean - risk_free_rate)/std
        sharpe_ratios.append(sharpe_ratio) 
        
    index = np.argmax(sharpe_ratios)
    
    return sharpe_ratios[index], portfolio_weights[index]

In [11]:
Prices2019 = get_adj_closing_prices(tickers, datetime.datetime(2018, 12, 31), datetime.datetime(2019, 3, 29))

# Jan

In [12]:
# get daily price data from 2018 Dec
prices = get_adj_closing_prices(tickers, datetime.datetime(2018, 12, 1), datetime.datetime(2018, 12, 31))

In [15]:
JanStart = Prices2019['2018-12-31':'2018-12-31']
JanEnd = Prices2019['2019-01-31':'2019-01-31']

In [16]:
daily_returns = prices.pct_change().dropna()

In [17]:
exp_returns_daily = compute_expected_daily_returns(daily_returns)
# convert average daily returns to annualized returns
exp_returns_monthly = np.power(1 + exp_returns_daily, 20) - 1

In [18]:
cov_daily = compute_daily_covariance_matrix(daily_returns)
cov_matrix_monthly = cov_daily * 20 # monthly: multiply with number of trading days in a month

In [19]:
print("Monthly Covariance Matrix")
print(' ' * 7, end="")   # 不换行输出
for tick in tickers:
    print("% 7s" % tick, end='')
print('\n', end="") #\n表示换行
print(' ' * 5, end="")
print('-' * 72)
for i, row_ticker in enumerate(tickers): #enumerate同时列出数据下标和数据
    print('%-4s | ' % row_ticker, end='')
    for j, col_ticker in enumerate(tickers):
        print("% 6.4f" % cov_matrix_monthly[i,j], end='')
    print('\n', end='')

Monthly Covariance Matrix
           NKE    JPM   SBUX  BRK-B    RRC    CAR   COST
     ------------------------------------------------------------------------
NKE  |  0.0228 0.0063 0.0065 0.0084 0.0155 0.0068 0.0092
JPM  |  0.0063 0.0065 0.0044 0.0067 0.0088 0.0076 0.0052
SBUX |  0.0065 0.0044 0.0055 0.0049 0.0101 0.0064 0.0073
BRK-B |  0.0084 0.0067 0.0049 0.0077 0.0097 0.0085 0.0067
RRC  |  0.0155 0.0088 0.0101 0.0097 0.0309 0.0104 0.0143
CAR  |  0.0068 0.0076 0.0064 0.0085 0.0104 0.0224 0.0094
COST |  0.0092 0.0052 0.0073 0.0067 0.0143 0.0094 0.0164


In [20]:
portfolio_weights = efficient_frontier(exp_returns_monthly, cov_matrix_monthly)

In [21]:
min_variance_weights = min_variance_portfolio(cov_matrix_monthly, portfolio_weights)
min_variance_expected_return = compute_portfolio_expected_return(exp_returns_monthly, min_variance_weights)
min_variance_std = compute_portfolio_std(cov_matrix_monthly, min_variance_weights)

In [22]:
for tick, x in zip(tickers, min_variance_weights):
    print('Weight %-4s %5.2f%%' % (tick, (x + 1e-10) * 100 ))

Weight NKE   0.00%
Weight JPM  34.68%
Weight SBUX 65.32%
Weight BRK-B  0.00%
Weight RRC   0.00%
Weight CAR   0.00%
Weight COST  0.00%


In [23]:
JanReturn = np.dot(JanEnd, (min_variance_weights/JanStart).T)
JanReturnRate = (JanReturn - 1)/1
print('Jan Return rate of min_variance: %5.2f%% ' % ((JanReturnRate + 1e-10) * 100) )

Jan Return rate of min_variance:  6.18% 


In [24]:
max_sharpe_ratio, max_sharpe_weights = max_sharpe_portfolio(exp_returns_monthly, cov_matrix_monthly, risk_free_rate, portfolio_weights)
max_sharpe_expected_return = compute_portfolio_expected_return(exp_returns_monthly, max_sharpe_weights)
max_sharpe_std = compute_portfolio_std(cov_matrix_monthly, max_sharpe_weights)

In [25]:
for tick, x in zip(tickers, max_sharpe_weights):
    print('Weight %-4s %5.2f%%' % (tick, (x + 1e-10) * 100 ))

Weight NKE  100.00%
Weight JPM   0.00%
Weight SBUX  0.00%
Weight BRK-B  0.00%
Weight RRC   0.00%
Weight CAR   0.00%
Weight COST  0.00%


In [26]:
JanReturn = np.dot(JanEnd, (max_sharpe_weights/JanStart).T)
JanReturnRate = (JanReturn - 1)/1
print('Jan Return rate of max_sharpe: %5.2f%% ' % ((JanReturnRate + 1e-10) * 100) )

Jan Return rate of max_sharpe: 10.44% 


# Feb

In [29]:
# get daily price data from 2019 Jan
prices = get_adj_closing_prices(tickers, datetime.datetime(2019, 1, 1), datetime.datetime(2019, 1, 30))

In [30]:
prices

,NKE,JPM,SBUX,BRK-B,RRC,CAR,COST
Date,,,,,,,
2019-01-02,73.869942,97.761940,63.690594,202.800003,9.941573,22.709999,203.658432
2019-01-03,72.563309,96.372574,60.927891,191.660004,9.931591,21.780001,199.341782
2019-01-04,74.458435,99.925385,62.947933,195.199997,10.670221,23.129999,205.130478
2019-01-07,75.525688,99.994858,62.947933,196.910004,11.109408,23.930000,205.886368
2019-01-08,76.533096,99.806297,62.997440,196.309998,10.979649,24.110001,207.428040
2019-01-09,76.393448,99.637589,63.254898,196.369995,11.578539,25.059999,208.770767
2019-01-10,76.223885,99.627663,63.561867,196.580002,11.418834,25.570000,209.506805
2019-01-11,75.844864,99.151314,63.106365,196.289993,11.478724,26.010000,209.377487
2019-01-14,75.894730,100.173492,62.749889,195.779999,11.848039,26.040001,207.875610


In [31]:
FebStart = Prices2019['2019-02-01':'2019-02-01']
FebEnd = Prices2019['2019-02-28':'2019-02-28']

In [32]:
daily_returns = prices.pct_change().dropna()

In [33]:
exp_returns_daily = compute_expected_daily_returns(daily_returns)
# convert average daily returns to annualized returns
exp_returns_monthly = np.power(1 + exp_returns_daily, 20) - 1

In [34]:
cov_daily = compute_daily_covariance_matrix(daily_returns)
cov_matrix_monthly = cov_daily * 20 # monthly: multiply with number of trading days in a month

In [36]:
print("Monthly Covariance Matrix")
print(' ' * 7, end="")   # 不换行输出
for tick in tickers:
    print("% 7s" % tick, end='')
print('\n', end="") #\n表示换行
print(' ' * 5, end="")
print('-' * 72)
for i, row_ticker in enumerate(tickers): #enumerate同时列出数据下标和数据
    print('%-4s | ' % row_ticker, end='')
    for j, col_ticker in enumerate(tickers):
        print("% 6.4f" % cov_matrix_monthly[i,j], end='')
    print('\n', end='')

Monthly Covariance Matrix
           NKE    JPM   SBUX  BRK-B    RRC    CAR   COST
     ------------------------------------------------------------------------
NKE  |  0.0025 0.0015 0.0023 0.0024 0.0013 0.0028 0.0018
JPM  |  0.0015 0.0025 0.0020 0.0025 0.0048 0.0044 0.0014
SBUX |  0.0023 0.0020 0.0061 0.0039 0.0026 0.0037 0.0021
BRK-B |  0.0024 0.0025 0.0039 0.0055 0.0041 0.0065 0.0022
RRC  |  0.0013 0.0048 0.0026 0.0041 0.0242 0.0161 0.0011
CAR  |  0.0028 0.0044 0.0037 0.0065 0.0161 0.0190 0.0031
COST |  0.0018 0.0014 0.0021 0.0022 0.0011 0.0031 0.0022


In [37]:
portfolio_weights = efficient_frontier(exp_returns_monthly, cov_matrix_monthly)

In [38]:
min_variance_weights = min_variance_portfolio(cov_matrix_monthly, portfolio_weights)
min_variance_expected_return = compute_portfolio_expected_return(exp_returns_monthly, min_variance_weights)
min_variance_std = compute_portfolio_std(cov_matrix_monthly, min_variance_weights)

In [39]:
for tick, x in zip(tickers, min_variance_weights):
    print('Weight %-4s %5.2f%%' % (tick, (x + 1e-10) * 100 ))

Weight NKE  22.78%
Weight JPM  35.82%
Weight SBUX  0.00%
Weight BRK-B  0.00%
Weight RRC   0.00%
Weight CAR   0.00%
Weight COST 41.40%


In [40]:
FebReturn = np.dot(FebEnd, (min_variance_weights/FebStart).T)
FebReturnRate = (FebReturn - 1)/1
print('Feb Return rate of min_variance: %5.2f%% ' % ((FebReturnRate + 1e-10) * 100) )

Feb Return rate of min_variance:  3.13% 


In [41]:
max_sharpe_ratio, max_sharpe_weights = max_sharpe_portfolio(exp_returns_monthly, cov_matrix_monthly, risk_free_rate, portfolio_weights)
max_sharpe_expected_return = compute_portfolio_expected_return(exp_returns_monthly, max_sharpe_weights)
max_sharpe_std = compute_portfolio_std(cov_matrix_monthly, max_sharpe_weights)

In [42]:
for tick, x in zip(tickers, max_sharpe_weights):
    print('Weight %-4s %5.2f%%' % (tick, (x + 1e-10) * 100 ))

Weight NKE  86.59%
Weight JPM   0.00%
Weight SBUX  0.00%
Weight BRK-B  0.00%
Weight RRC   0.00%
Weight CAR  13.41%
Weight COST  0.00%


In [43]:
FebReturn = np.dot(FebEnd,(max_sharpe_weights/FebStart).T)
FebReturnRate = (FebReturn - 1)/1
print('Feb Return rate of max_sharpe: %5.2f%% ' % ((FebReturnRate + 1e-10) * 100) )

Feb Return rate of max_sharpe:  9.12% 


# Mar

In [45]:
# get daily price data from 1998 to 2019 Mar
prices = get_adj_closing_prices(tickers, datetime.datetime(2019, 2, 1), datetime.datetime(2019, 2, 27))
prices

,NKE,JPM,SBUX,BRK-B,RRC,CAR,COST
Date,,,,,,,
2019-02-01,81.300827,103.091156,67.443504,209.190002,10.929741,26.610001,209.138794
2019-02-04,81.779594,103.458351,66.918694,208.520004,10.869852,26.389999,210.043869
2019-02-05,82.647362,103.001846,68.661461,207.820007,10.700167,26.680000,210.471588
2019-02-06,82.497749,102.952225,68.532066,206.009995,10.121240,26.540001,210.461639
2019-02-07,82.178566,101.602547,68.820724,201.729996,9.642128,26.120001,208.486969
2019-02-08,82.148643,100.590302,69.427910,200.919998,9.422534,25.680000,207.190475
2019-02-11,83.465256,100.113937,69.666801,203.050003,10.330852,27.580000,208.157852
2019-02-12,84.492615,101.820877,69.427910,206.149994,10.390740,27.610001,211.329285
2019-02-13,85.180847,102.307159,70.184402,205.800003,10.370777,27.520000,212.226868


In [46]:
MarStart = Prices2019['2019-03-01':'2019-03-01']
MarEnd = Prices2019['2019-03-29':'2019-03-29']

In [47]:
daily_returns = prices.pct_change().dropna()

In [48]:
exp_returns_daily = compute_expected_daily_returns(daily_returns)
# convert average daily returns to annualized returns
exp_returns_monthly = np.power(1 + exp_returns_daily, 20) - 1

In [49]:
cov_daily = compute_daily_covariance_matrix(daily_returns)
cov_matrix_monthly = cov_daily * 20 # monthly: multiply with number of trading days in a month

In [50]:
print("Monthly Covariance Matrix")
print(' ' * 7, end="")   # 不换行输出
for tick in tickers:
    print("% 7s" % tick, end='')
print('\n', end="") #\n表示换行
print(' ' * 5, end="")
print('-' * 72)
for i, row_ticker in enumerate(tickers): #enumerate同时列出数据下标和数据
    print('%-4s | ' % row_ticker, end='')
    for j, col_ticker in enumerate(tickers):
        print("% 6.4f" % cov_matrix_monthly[i,j], end='')
    print('\n', end='')

Monthly Covariance Matrix
           NKE    JPM   SBUX  BRK-B    RRC    CAR   COST
     ------------------------------------------------------------------------
NKE  |  0.0012 0.0006 0.0002 0.0007 0.0022-0.0015 0.0005
JPM  |  0.0006 0.0022-0.0005 0.0014 0.0032-0.0002 0.0008
SBUX |  0.0002-0.0005 0.0015-0.0005-0.0006 0.0003-0.0003
BRK-B |  0.0007 0.0014-0.0005 0.0018 0.0044 0.0010 0.0007
RRC  |  0.0022 0.0032-0.0006 0.0044 0.0272 0.0055 0.0021
CAR  | -0.0015-0.0002 0.0003 0.0010 0.0055 0.0387-0.0010
COST |  0.0005 0.0008-0.0003 0.0007 0.0021-0.0010 0.0008


In [51]:
portfolio_weights = efficient_frontier(exp_returns_monthly, cov_matrix_monthly)

In [52]:
min_variance_weights = min_variance_portfolio(cov_matrix_monthly, portfolio_weights)
min_variance_expected_return = compute_portfolio_expected_return(exp_returns_monthly, min_variance_weights)
min_variance_std = compute_portfolio_std(cov_matrix_monthly, min_variance_weights)

In [53]:
for tick, x in zip(tickers, min_variance_weights):
    print('Weight %-4s %5.2f%%' % (tick, (x + 1e-10) * 100 ))

Weight NKE   0.97%
Weight JPM   0.24%
Weight SBUX 36.57%
Weight BRK-B  8.10%
Weight RRC   0.00%
Weight CAR   1.97%
Weight COST 52.16%


In [54]:
MarReturn = np.dot(MarEnd, (min_variance_weights/MarStart).T)
MarReturnRate = (MarReturn - 1)/1
print('Mar Return rate of min_variance: %5.2f%% ' % ((MarReturnRate + 1e-10) * 100) )

Mar Return rate of min_variance:  6.99% 


In [55]:
max_sharpe_ratio, max_sharpe_weights = max_sharpe_portfolio(exp_returns_monthly, cov_matrix_monthly, risk_free_rate, portfolio_weights)
max_sharpe_expected_return = compute_portfolio_expected_return(exp_returns_monthly, max_sharpe_weights)
max_sharpe_std = compute_portfolio_std(cov_matrix_monthly, max_sharpe_weights)

In [56]:
for tick, x in zip(tickers, max_sharpe_weights):
    print('Weight %-4s %5.2f%%' % (tick, (x + 1e-10) * 100 ))

Weight NKE  36.50%
Weight JPM   0.00%
Weight SBUX 11.51%
Weight BRK-B  0.00%
Weight RRC   0.00%
Weight CAR  15.43%
Weight COST 36.56%


In [57]:
MarReturn = np.dot(MarEnd,(max_sharpe_weights/MarStart).T)
MarReturnRate = (MarReturn - 1)/1
print('Mar Return rate of max_sharpe: %5.2f%% ' % ((MarReturnRate + 1e-10) * 100) )

Mar Return rate of max_sharpe:  2.51% 
